# VacationPy
----

In [1]:
#dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from config import gkey

### Store Part I results into DataFrame

In [2]:
#create dataframe of city weather from csv
city_weather_df = pd.read_csv("output/citiesweather.csv", index_col = 0)

#preview frame
city_weather_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.1203,-134.9692,74.30,75,100,26.55,PF,1623976991
1,Norman Wells,65.2820,-126.8329,78.84,27,40,11.50,CA,1623977016
2,Bethel,41.3712,-73.4140,70.75,58,1,3.44,US,1623976999
3,Faanui,-16.4833,-151.7500,80.15,69,17,17.72,PF,1623977016
4,Bāndarban,22.1965,92.2262,74.14,96,100,2.64,BD,1623977016


### Humidity Heatmap

In [3]:
#configure gmaps
gmaps.configure(api_key=gkey)

#customize the size of the figure
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

#define locations of interest and level of humidity
locations = city_weather_df[["Latitude", "Longitude"]]
humid = city_weather_df["Humidity"]

#build heatlayer
humidity_layer = gmaps.heatmap_layer(locations, weights = humid, 
                                 dissipating = True, max_intensity = 100,
                                 point_radius = 15)

#add heatlayer layer to map
fig.add_layer(humidity_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria

In [4]:
#create dataframe of cities that fit weather criteria
ideal_weather_df = city_weather_df.loc[(city_weather_df['Max Temperature'] > 70) & (city_weather_df['Max Temperature'] < 80) & (city_weather_df['Wind Speed'] < 10) & (city_weather_df['Cloudiness'] == 0),  :]

#view frame
ideal_weather_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
10,Hambantota,6.1241,81.1185,77.34,100,0,5.75,LK,1623977018
179,Cayenne,4.9333,-52.3333,77.09,90,0,2.30,GF,1623977266
192,Buraidah,26.3260,43.9750,77.45,20,0,9.46,SA,1623977335
195,Verkhoyansk,67.5447,133.3850,76.95,27,0,5.19,RU,1623977336
350,Zhanakorgan,43.9098,67.2495,78.44,17,0,4.21,KZ,1623977572
353,Meyzieu,45.7667,5.0000,75.15,56,0,3.44,FR,1623977573
400,Waddān,29.1614,16.1390,75.81,48,0,9.40,LY,1623977650


## Hotel Map

In [5]:
#store into new dataframe named hotel_df
hotel_df = ideal_weather_df

#create hotel name column within dataframe
hotel_df['Hotel Name'] = ''

#set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = { 
    "radius": 5000,
    "type": "lodging",
    "keyword": "hotel",
    "key": gkey,
}
#iterate through pandas dataframe
for index, row in hotel_df.iterrows():

#define city and location
    lat = row['Latitude']
    lng = row['Longitude']
    city = row['City']
    
#set location variable within call specifications
    target_coordinates = f'{lat}, {lng}'

#add location parameter
    params['location'] = target_coordinates

    print(f"Retrieving Results for {city}.")

#assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
#extract results
    results = response['results']
    
#add found hotel to dataframe
    try:
        print(f"Closest hotel to {city} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
#add 'none' to dataframe, if no hotel found or error
    except (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "None"

        print("Missing field/result or no hotel within range.")
        
    print("------------")

<ipython-input-5-f21582b3a30d>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''
Retrieving Results for Hambantota.
C:\Users\terre\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
Missing field/result or no hotel within range.
------------
Retrieving Results for Cayenne.
Closest hotel to Cayenne is Mercure Cayenne Royal Amazonia.
------------
Retrieving Results for Buraidah.
Closest hotel to Buraidah is Möv

In [6]:
#create new dataframe of only cities with hotels withing 5000m of coordinates
hotel_clean_df = hotel_df.loc[hotel_df['Hotel Name'] != 'None', :]
hotel_clean_df

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
179,Cayenne,4.9333,-52.3333,77.09,90,0,2.30,GF,1623977266,Mercure Cayenne Royal Amazonia
192,Buraidah,26.3260,43.9750,77.45,20,0,9.46,SA,1623977335,Mövenpick Hotel Qassim
350,Zhanakorgan,43.9098,67.2495,78.44,17,0,4.21,KZ,1623977572,"""Нұрлан"" қонақ үйі"
353,Meyzieu,45.7667,5.0000,75.15,56,0,3.44,FR,1623977573,Gatsby Hotel & Restaurant By Happyculturetm


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_clean_info = [info_box_template.format(**row) for index, row in hotel_clean_df.iterrows()]
locations = hotel_clean_df[["Latitude", "Longitude"]]

In [8]:
#build marker layer
markers = gmaps.marker_layer(locations, info_box_content= hotel_clean_info)

#add layer to figure with heatmap
fig.add_layer(markers)

#display
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…